# State space labelling in pyGSTi

Instances of `pygsti.objects.StateSpaceLabels` describe the structure of a model's state space and associate labels with the parts of that structure.  This is particularly useful when dealing with multiple qubits or a qubit and its environment, as it can be useful to reference subspaces or subsystems of the entire quantum state space.

In general, a state space is the direct sum of one or more *tensor product blocks*, each of which is the tensor product of one or more *factors*: 

$$ \mbox{State space} = (\mathcal{H}_1^A \otimes \mathcal{H}_2^A \otimes \cdots) \oplus (\mathcal{H}_1^B \otimes \mathcal{H}_2^B \otimes \cdots) \oplus \cdots$$

In the above expression the tensor product blocks are in parenthesis and labelled by $A$, $B$, etc., and the $\mathcal{H}_i^X$ are the factors.  We can initialize a `StateSpaceLabels` object using a list of tuples containing labels and dimensions which mirror this structure, i.e.

~~~
StateSpaceLabels( [(H1A_label, H2A_label, ...), ((H1B_label, H2B_label, ...), ... ],
                  [(H1A_dim  , H2A_dim, ...),   ((H1B_dim  , H2B_dim, ...), ... ])
~~~
where `_label` variables are either strings or integers (it can be convenient to label qubits, for instance, by integers) and `_dim` variables are always integers.  Here are some examples:

In [ ]:
import pygsti
from pygsti.objects import StateSpaceLabels

lbls = StateSpaceLabels([('H0','H1')], [(2,3)])
print(lbls) # label(dim) notation, '*' means 'otimes', '+' means 'oplus'

lbls2 = StateSpaceLabels(('H0','H1'), (2,3)) # same as above - a *single* tensor product block
print(lbls2)

lbls3 = StateSpaceLabels([('H0',), ('H1',)], [(2,),(3,)]) # direct sum
print(lbls3)

lbls4 = StateSpaceLabels([('H1a','H2a'), ('H1b','H2b')], [(2,1),(3,4)])
print(lbls4)

Since we're often dealing with qubits (dimension = 2 factors), the labels beginning with 'Q' or that are integers default to dimension 2.  Similarly, labels beginning with 'L' default dimension 1 (an additional "Level").  If all the labels in the first argument passed to the `StateSpaceLabels` constructor have defaults, then the **second argument (the dimensions) may be omitted**.  For example:

In [ ]:
lbls5 = StateSpaceLabels(['Q0','Q1']) # 2 qubits
print(lbls5)

lbls6 = StateSpaceLabels(list(range(3))) # 3 qubits
print(lbls6)

lbls7 = StateSpaceLabels([('Q0','Q1'),('Leakage',)])
print(lbls7)

The raw data within a `StateSpaceLabels` object is stored in the `.labels` and `.labeldims` members (but be careful, `labeldims` is a *dictionary*):

In [ ]:
lbls7.labels

In [ ]:
lbls7.labeldims

You can access the total dimension of the state space using the `.dim` member ($d^2$ if the density-matrix dimension is $d$, i.e. when density matrices are $d \times d$), and the tensor-product block dimensions $k_i^2$ (there is a kite structure of are $k_i\times k_i$ nonzero blocks that form  the the $d \times d$ density matrix, so $\sum_i k_i = d$) via the `.tpb_dims` property.  You can also access the per-label dimension via the `.labeldims` dictionary.

In [ ]:
print("Total dim = ",lbls7.dim)
#print("Separately: ",lbls7.dim, lbls7.dim.blockDims, lbls7.dim.opDim, lbls7.dim.embedDim)
print("Dimensions of tensor product blocks = ", lbls7.tpb_dims)
print("Dimension of the space associated with each label: ", lbls7.labeldims)

There are also few convenience functions that make it easier to access the raw data: 


In [ ]:
print("Number of tensor product blocks = ",lbls7.num_tensor_prod_blocks())
print("The labels in the 0th tensor product block are: ",lbls7.tensor_product_block_labels(0))
print("The dimensions corresponding to those labels are: ",lbls7.tensor_product_block_dims(0))
print("The 'Q0' labels exists in the tensor product block w/index=",lbls7.tpb_index['Q0'])
print("The product of the dimensions associated with 'Q0' and 'Leakage' = ",lbls7.product_dim( ('Q0','Leakage') ))

**That's it!** You know all there is to know about the `StateSpaceLabels` object.  Remember you can pass a `StateSpaceLabels` object to `pygsti.construction.create_explicit_model` to create a model which operates on the given state space.